In [1]:
import os
import argparse
import multiprocessing
from collections import Counter

import torch
import torchaudio
import numpy as np
import pandas as pd

from models import PretrainedModel, Model
from data import SLUDataset, get_SLU_datasets, read_config
from training import Trainer

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
config_path = 'unfreeze_word_layers.cfg'
config = read_config(config_path)
torch.manual_seed(config.seed); np.random.seed(config.seed)

# 1. Generate datasets

In [3]:
train_dataset, valid_dataset, test_dataset = get_SLU_datasets(config)
#train_dataset_sample = next(iter(train_dataset.loader))

In [7]:
config.Sy_intent

{'action': {'change language': 0,
  'activate': 1,
  'deactivate': 2,
  'increase': 3,
  'decrease': 4,
  'bring': 5},
 'object': {'none': 0,
  'music': 1,
  'lights': 2,
  'volume': 3,
  'heat': 4,
  'lamp': 5,
  'newspaper': 6,
  'juice': 7,
  'socks': 8,
  'Chinese': 9,
  'Korean': 10,
  'English': 11,
  'German': 12,
  'shoes': 13},
 'location': {'none': 0, 'kitchen': 1, 'bedroom': 2, 'washroom': 3}}

# 2. Train SLU Model

In [7]:
# Initialize SLU Model
model = Model(config=config)
model

Model(
  (pretrained_model): PretrainedModel(
    (phoneme_layers): ModuleList(
      (0): SincLayer()
      (1): Abs()
      (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (3): LeakyReLU(negative_slope=0.2)
      (4): Dropout(p=0.0, inplace=False)
      (5): Conv1d(80, 60, kernel_size=(5,), stride=(1,), padding=(2,))
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=True)
      (7): LeakyReLU(negative_slope=0.2)
      (8): Dropout(p=0.0, inplace=False)
      (9): Conv1d(60, 60, kernel_size=(5,), stride=(1,), padding=(2,))
      (10): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=True)
      (11): LeakyReLU(negative_slope=0.2)
      (12): Dropout(p=0.0, inplace=False)
      (13): NCL2NLC()
      (14): GRU(60, 128, batch_first=True, bidirectional=True)
      (15): RNNSelect()
      (16): Dropout(p=0.5, inplace=False)
      (17): Downsample()
      (18): GRU(256, 128, batch_first=True, bidirectional=

In [8]:
%%time

# Train the model
trainer = Trainer(model=model, config=config)
# if restart: trainer.load_checkpoint()

for epoch in range(config.training_num_epochs):
    print("========= Epoch %d of %d =========" % (epoch+1, config.training_num_epochs))
    train_intent_acc, train_intent_loss = trainer.train(train_dataset)
    valid_intent_acc, valid_intent_loss = trainer.test(valid_dataset)

    print("========= Results: epoch %d of %d =========" % (epoch+1, config.training_num_epochs))
    print("*intents*| train accuracy: %.2f| train loss: %.2f| valid accuracy: %.2f| valid loss: %.2f\n" % (train_intent_acc, train_intent_loss, valid_intent_acc, valid_intent_loss) )
    
    trainer.save_checkpoint()

  0%|          | 0/723 [00:00<?, ?it/s]

========= Epoch 1 of 20 =========
sinc0: frozen
conv1: frozen
conv2: frozen
phone_rnn0: frozen
phone_rnn1: frozen
word_rnn0: frozen
word_rnn1: frozen


  0%|          | 1/723 [00:00<10:20,  1.16it/s]

intent loss: 5.902858734130859
intent acc: 0.0


 14%|█▍        | 101/723 [00:20<02:05,  4.97it/s]

...

intent loss: 0.023221246898174286
intent acc: 0.984375


 97%|█████████▋| 701/723 [02:25<00:04,  4.75it/s]

intent loss: 0.05376802384853363
intent acc: 0.96875


100%|██████████| 723/723 [02:30<00:00,  5.10it/s]


========= Results: epoch 20 of 20 =========
*intents*| train accuracy: 0.98| train loss: 0.06| valid accuracy: 0.93| valid loss: 0.40

CPU times: user 45min 35s, sys: 6min 52s, total: 52min 28s
Wall time: 52min 39s


# 3. Evaluate the Model

In [9]:
test_intent_acc, test_intent_loss = trainer.test(test_dataset)
print("========= Test results =========")
print("*intents*| test accuracy: %.2f| test loss: %.2f| valid accuracy: %.2f| valid loss: %.2f\n" % (test_intent_acc, test_intent_loss, valid_intent_acc, valid_intent_loss) )

========= Test results =========
*intents*| test accuracy: 0.99| test loss: 0.05| valid accuracy: 0.93| valid loss: 0.40

